<a href="https://colab.research.google.com/github/fryall/fofo/blob/main/NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import os,glob
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # import was correct
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Dense, InputLayer


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
image_size=[224,224]
input_folder = '/content/drive/MyDrive/try1'
splitfolders.ratio(input_folder, output="twenty",
                   seed=42, ratio=(.7, .3 ),
                   group_prefix=None)

Copying files: 6271 files [01:56, 54.00 files/s] 


In [50]:
import tensorflow as tf

# Define the paths to the train and validation directories
train_dir = 'twenty/train'
val_dir = 'twenty/val'

# Load the training data
train_path = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # same as your image_size
    batch_size=32,          # adjust this as needed
    label_mode='categorical'  # or 'binary' depending on your labels
)

# Load the validation data
test_path= tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),  # same as your image_size
    batch_size=32,          # adjust this as needed
    label_mode='categorical'  # or 'binary' depending on your labels
)

Found 4380 files belonging to 20 classes.
Found 1889 files belonging to 20 classes.


In [51]:
from keras.applications import VGG16
VGG=VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')

In [52]:
VGG.trainable=False

In [53]:
input_shape = (224, 224, 3)

In [54]:
model=Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(VGG)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=300,activation="relu"))
model.add(Dense(units=300,activation="relu"))
model.add(Dense(units=20,activation="softmax"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [55]:
print(VGG.output_shape)

(None, 7, 7, 512)


In [56]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 300)                 │         153,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 20)                  │           6,020 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,964,908 (57.09 MB)

 Trainable params: 250,220 (977.42 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [58]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [59]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.keras", monitor="val_accuracy", verbose=1,
                             save_best_only=True, save_weights_only=False)
# Removed 'early' parameter as it's not part of ModelCheckpoint

early_stopping = EarlyStopping(monitor="val_accuracy", min_delta=0, patience=40,
                               verbose=1, mode='auto')
# Created a separate EarlyStopping callback

In [ ]:
history = model.fit(x=train_path, steps_per_epoch=137, epochs=10,
                 validation_data=test_path, validation_steps=1,
                 callbacks=[checkpoint])

Epoch 1/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.3368 - loss: 3.2131 
Epoch 1: val_accuracy improved from -inf to 0.43750, saving model to vgg16_1.keras
137/137 ━━━━━━━━━━━━━━━━━━━━ 2906s 21s/step - accuracy: 0.3376 - loss: 3.2055 - val_accuracy: 0.4375 - val_loss: 2.1643
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy improved from 0.43750 to 0.68750, saving model to vgg16_1.keras
137/137 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 1.0410
Epoch 3/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.7055 - loss: 0.9891 
Epoch 3: val_accuracy did not improve from 0.68750
137/137 ━━━━━━━━━━━━━━━━━━━━ 2884s 21s/step - accuracy: 0.7056 - loss: 0.9888 - val_accuracy: 0.6562 - val_loss: 1.2093
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.68750
137/137 ━━━━━━━━━━━━━━━━━━━━ 22s 158ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6562 - val_loss: 1.1730
Epoch 5/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.8229 - loss: 0.5894 
Epoch 5: val_accuracy did not improve from 0.68750
137/137 ━━━━━━━━━━━━━━━━━━━━ 2944s 21s/step - accuracy: 0.8229 - loss: 0.5893 - val_accuracy: 0.6562 - val_loss: 1.3076
Epoch 6/10

Epoch 6: val_accuracy improved from 0.68750 to 0.84375, saving mod